# TensorFlow practice 1

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [15]:
import tensorflow as tf
import datetime
import pandas as pd
import numpy as np

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

11493376/11490434 [==============================] - 3s 0us/step


In [5]:
# this why input_shap=(28, 28)
x_train.shape

(60000, 28, 28)

In [10]:
x_train[0,0,]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
# find out the unique values
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [4]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2225 - accuracy: 0.9341 - val_loss: 0.1018 - val_accuracy: 0.9690
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0976 - accuracy: 0.9699 - val_loss: 0.0740 - val_accuracy: 0.9756
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0690 - accuracy: 0.9779 - val_loss: 0.0674 - val_accuracy: 0.9792
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0545 - accuracy: 0.9826 - val_loss: 0.0644 - val_accuracy: 0.9794
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0430 - accuracy: 0.9860 - val_loss: 0.0690 - val_accuracy: 0.9796


In [18]:
%tensorboard --logdir logs/fit

## Using TensorBoard with other methods

In [19]:
#Use the same dataset as above, but convert it to tf.data.Dataset to take advantage of batching capabilities:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [20]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [21]:
#Create stateful metrics that can be used to accumulate values during training and logged at any point:

# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [22]:
#Define the training and test functions:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        train_loss(loss)
        train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [23]:
#Set up summary writers to write the summaries to disk in a different logs directory:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [25]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))
    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()


Epoch 1, Loss: 0.11958672106266022, Accuracy: 96.41278076171875, Test Loss: 0.08231931179761887, Test Accuracy: 97.413330078125
Epoch 2, Loss: 0.08598058670759201, Accuracy: 97.38833618164062, Test Loss: 0.07157903909683228, Test Accuracy: 97.70999908447266
Epoch 3, Loss: 0.059527814388275146, Accuracy: 98.1816635131836, Test Loss: 0.06384428590536118, Test Accuracy: 98.04000091552734
Epoch 4, Loss: 0.04597246274352074, Accuracy: 98.57333374023438, Test Loss: 0.06689930707216263, Test Accuracy: 97.80999755859375
Epoch 5, Loss: 0.03637506440281868, Accuracy: 98.85832977294922, Test Loss: 0.05858888477087021, Test Accuracy: 98.02999877929688


In [26]:
%tensorboard --logdir logs/gradient_tape